<a href="https://colab.research.google.com/github/ajafarsadiq2002/Fake-News-Classifier-Using-LSTM/blob/main/Fake_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**HOW TO IMPORT DATA SETS FROM KAGGLE ,STEPS TO BE FOLLOWED ARE GIVEN DOWN**

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ajafarsadiq2002","key":"516e140961b3d4b26dd1d635bd82f0ef"}'}

In [3]:
!mkdir ~/.kaggle

In [4]:
!mv kaggle.json ~/.kaggle/


In [5]:
!chmod 600 ~/.kaggle/kaggle.json


In [6]:
! kaggle datasets list


ref                                                       title                                        size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  ------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
sazidthe1/data-science-salaries                           Data Science Salaries 2024                   57KB  2024-01-20 16:45:27            844         26  1.0              
michau96/social-media-popularity-2009-2023                Social media popularity (2009 - 2023)         5KB  2024-01-16 16:45:09            694         21  0.9411765        
hummaamqaasim/jobs-in-data                                Jobs and Salaries in Data Science            76KB  2023-12-25 20:03:32           8112        165  1.0              
nelgiriyewithana/apple-quality                            Apple Quality                               170KB  2024-01-11 14:31:07  

In [7]:
! kaggle competitions download -c fake-news

403 - Forbidden - You must accept this competition's rules before you'll be able to download files.


In [8]:
! kaggle competitions download -c fake-news

 88% 41.0M/46.5M [00:00<00:00, 113MB/s] 
100% 46.5M/46.5M [00:00<00:00, 111MB/s]


In [9]:
!unzip fake-news.zip


Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


**Fake News Classifier Using LSTM**


In [10]:
import pandas as pd


In [13]:
df=pd.read_csv('train.csv')


In [14]:
df.head()


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
###Drop Nan Values
df=df.dropna()

In [16]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [17]:
## Get the Dependent features
y=df['label']


In [19]:
X.shape

(18285, 4)

In [20]:
y.shape

(18285,)

In [21]:
import tensorflow as tf


In [22]:
tf.__version__

'2.15.0'

**IMPORTING LIBRARIES**

In [23]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [24]:
### Vocabulary size
voc_size=5000

**ONE HOT REPRESENTATION**

In [25]:
messages=X.copy()

In [30]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [26]:
messages.reset_index(inplace=True)

In [27]:
import nltk
import re
from nltk.corpus import stopwords

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [31]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [32]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[236, 3920, 496, 1835, 4217, 1829, 1228, 2361, 3413, 4072],
 [290, 3568, 1959, 4510, 641, 2638, 748],
 [3343, 1783, 1470, 1792],
 [4037, 3795, 4473, 3727, 2851, 601],
 [3430, 641, 4695, 2395, 2611, 4335, 641, 578, 3155, 4860],
 [2289,
  2903,
  1949,
  1697,
  4589,
  1133,
  3969,
  4745,
  4163,
  3408,
  3637,
  4255,
  3496,
  656,
  748],
 [825, 1539, 4042, 127, 3818, 392, 355, 4126, 2972, 4955, 3414],
 [4030, 911, 3705, 221, 1410, 1271, 1133, 3001, 2972, 4955, 3414],
 [2945, 3310, 4823, 1294, 53, 1902, 1053, 922, 1133, 6],
 [4759, 907, 4623, 149, 2801, 578, 3791, 2297],
 [3666, 3416, 1274, 386, 406, 3330, 3063, 4539, 2324, 4525, 1998],
 [3727, 1360, 4217, 1902, 1133, 1410],
 [834, 938, 2662, 3463, 4903, 1244, 2711, 2851, 4259],
 [3071, 33, 1159, 1020, 4572, 4251, 3872, 2972, 4955, 3414],
 [3390, 2175, 2616, 4032, 85, 2972, 4955, 3414],
 [1081, 4506, 627, 3078, 3945, 2308, 875, 559, 2931, 169],
 [4403, 2882, 3568],
 [3940, 4604, 1415, 4807, 1133, 4613, 210, 748],
 [49, 1222, 1959

**EMBEDDING REPRESENTATION**

In [33]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2361 3413 4072]
 [   0    0    0 ...  641 2638  748]
 [   0    0    0 ... 1783 1470 1792]
 ...
 [   0    0    0 ... 2972 4955 3414]
 [   0    0    0 ... 2945 4564 3778]
 [   0    0    0 ... 3551 2005 4908]]


In [34]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  236,
       3920,  496, 1835, 4217, 1829, 1228, 2361, 3413, 4072], dtype=int32)

In [35]:
len(embedded_docs)

18285

In [36]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [37]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [38]:
len(embedded_docs),y.shape

(18285, (18285,))

In [39]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [40]:
X_final.shape,y_final.shape


((18285, 20), (18285,))

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

**Adding Dropout**


In [46]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [47]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 44ms/step - loss: 0.3436 - accuracy: 0.8369 - val_loss: 0.1951 - val_accuracy: 0.9147
Epoch 2/10
192/192 [==============================] - 8s 44ms/step - loss: 0.1459 - accuracy: 0.9411 - val_loss: 0.1902 - val_accuracy: 0.9211
Epoch 3/10
192/192 [==============================] - 9s 48ms/step - loss: 0.1032 - accuracy: 0.9601 - val_loss: 0.2495 - val_accuracy: 0.9211
Epoch 4/10
192/192 [==============================] - 8s 39ms/step - loss: 0.0788 - accuracy: 0.9717 - val_loss: 0.2195 - val_accuracy: 0.9186
Epoch 5/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0565 - accuracy: 0.9800 - val_loss: 0.2736 - val_accuracy: 0.9097
Epoch 6/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0390 - accuracy: 0.9864 - val_loss: 0.3192 - val_accuracy: 0.9163
Epoch 7/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0307 - accuracy: 0.9886 - val_loss: 0.3830 - val_accuracy: 0.912

**Performance Metrics And Accuracy**


In [50]:
y_pred=(model.predict(X_test) > 0.5)


189/189 [==============================] - 1s 7ms/step


In [51]:
from sklearn.metrics import confusion_matrix


In [52]:
confusion_matrix(y_test,y_pred)


array([[3095,  324],
       [ 198, 2418]])

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9135045567522784